# Recherche

On peut commencer par faire un rechercher remplacer pour transformer un nombre en notation scientifique en équivalent LaTeX. Même si maintenant ce n'est plus indispensable, il est préférable de compiler l'expression régulière pour gagner en temps d'exécution par la suite.

In [12]:
import re

text = "7.34e-06"
replaceSciLatex = re.compile(r"""
(\d*)   # On capture la partie avant le séparateur décimal
\.      #Le séparateur décimal
(\d*)   #on récupère les décimales
e       #le séparateur des puissances
(-?)+?  #il peut y avoir un signe devant la puissance de 10, on ne veut capturer que le signe -
0?      #s'il y a un zéro, on ne veut pas le capturer
(\d*)   #on capture la puissance de 10
""",flags=re.VERBOSE)
answer = replaceSciLatex.search(text)
#On va afficher tous les groupes capturés
for catch in answer.groups():
    print(catch)

#version équivalente non compilée :
answer = re.search(r"""
(\d*)   # On capture la partie avant le séparateur décimal
\.      #Le séparateur décimal
(\d*)   #on récupère les décimales
e       #le séparateur des puissances
([-+])? #il peut y avoir un signe devant la puissance de 10
0?      #s'il y a un zéro, on ne veut pas le capturer
(\d*)   #on capture la puissance de 10
""",text,flags=re.VERBOSE)
for catch in answer.groups():
    print(catch)

7
34
-
6
7
34
-
6


# Remplacement

Maintenant, on va faire la transformation pour passer en notation LaTeX

In [16]:
text = "7.34e-06"

replaceSciLatex = re.compile(r"(\d*)\.(\d*)e(-?)+?0?(\d*)")
#Les \1, \2, \3, \4 font références aux groupes capturés auparavant
answer = replaceSciLatex.sub(r"\1,\2\\cdot 10^{\3\4}",text)
if answer :
    print('match')
    print(answer)
else :
    print('nomatch')

match
7,34\cdot 10^{-6}


# Nécessité des expressions régulières

On pourrait se dire qu'il est possible de faire pareil sans expressions régulières, mais cela prendrait plus de temps à programmer. Et il est plus difficile de prendre en compte les cas où la puissance de 10 contient un zéro, tout en traitant les cas où la puissance est positive ou non.

In [18]:
text = "7.34e-06"
#Premier essai, ce n'est pas bon si on a un zéro :
answer = text.replace(".",",").replace("e-","\cdot 10^{-").replace("e","\cdot 10^{")+"}"
print(answer)
#Il faut à chaque fois traiter différemment le cas d'une puissance positive et d'une puissance négative.
answer = answer.replace("^{0","^{").replace("^{-0","^{-")
print(answer)

7,34\cdot 10^{-06}
7,34\cdot 10^{-6}


Et maintenant, si on change le texte, on peut voir la différence entre les deux cas.

In [45]:
text = "La valeur de alpha vaut 7.34e-06 eV. Pourtant, si on avait pris 0.65e+10 comme valeur de beta, on aurait trouvé que alpha vaut maintenant 18,95 "
#on a au passage adapté pour les deux cas pour le séparateur décimal qui peut être un point ou une virgule
replaceSciLatex = re.compile(r"(\d*)[\.,](\d*)e(-?)+?0?(\d*)")

#Les \1, \2, \3, \4 font références aux groupes capturés auparavant
answer = replaceSciLatex.sub(r"$\1,\2\\cdot 10^{\3\4}$",text)
print("remplacement correct avec expression régulière")
print(answer)

print("\nremplacement incorrect avec rechercher/remplacer brut")
#Avec un rechercher remplacer :
answer = text.replace(".",",").replace("e-","\cdot 10^{-").replace("e","\cdot 10^{")+"}"
print(answer)
#Il faut à chaque fois traiter différemment le cas d'une puissance positive et d'une puissance négative.
answer = answer.replace("^{0","^{").replace("^{-0","^{-")


remplacement correct avec expression régulière
La valeur de alpha vaut $7,34\cdot 10^{-6}$ eV. Pourtant, si on avait pris $0,65\cdot 10^{}$+10 comme valeur de beta, on aurait trouvé que alpha vaut maintenant 18,95 

remplacement incorrect avec rechercher/remplacer brut
La val\cdot 10^{ur d\cdot 10^{ alpha vaut 7,34\cdot 10^{-06 \cdot 10^{V, Pourtant, si on avait pris 0,65\cdot 10^{+10 comm\cdot 10^{ val\cdot 10^{ur d\cdot 10^{ b\cdot 10^{ta, on aurait trouvé qu\cdot 10^{ alpha vaut maint\cdot 10^{nant 18,95 }


# Un exemple de conversion 
Sur cette base, on peut maintenant écrire très facilement un code pour transformer un tableau de type nparray en équivalent LaTeX :

In [48]:
#Création du tableau 
import numpy as np
array = np.random.normal(loc=0,scale=1,size=(4,4))
text = str(np.round(array,3))
print(text)
#Pour créer les espacement intermédiaires, il faut détecter ceux qui sont entre deux chiffres
#avec éventuellement un signe "-" devant, un simple rechercher/remplacer aurait ajouté des colonnes au début et à la fin
makeTable = re.compile(r"(\d)\s{1,}(-?\d)")
text1 = makeTable.sub(r"\1 & \2",text)
print(text1)
#Maintenant, un simple rechercher/remplacer suffit, donc on peut s'en contenter
text2 = text1.replace("[[","\\begin{pmatrix}\n ").replace("]]","\\\\\n\\end{pmatrix}").replace("[","").replace("]",r"\\")
print(text2)

[[-1.827  0.488  0.671  0.583]
 [ 0.317  0.993  1.974  2.716]
 [-1.625  0.03   0.143 -0.909]
 [-0.172  0.426 -1.332 -1.11 ]]
[[-1.827 & 0.488 & 0.671 & 0.583]
 [ 0.317 & 0.993 & 1.974 & 2.716]
 [-1.625 & 0.03 & 0.143 & -0.909]
 [-0.172 & 0.426 & -1.332 & -1.11 ]]
\begin{pmatrix}
 -1.827 & 0.488 & 0.671 & 0.583\\
  0.317 & 0.993 & 1.974 & 2.716\\
 -1.625 & 0.03 & 0.143 & -0.909\\
 -0.172 & 0.426 & -1.332 & -1.11 \\
\end{pmatrix}


Si jamais on a des chiffres en notation scientifique, on peut adapter le code très facilement avec les exemples précédents :

In [58]:
#Création du tableau 
import numpy as np
array = np.random.normal(loc=0,scale=1,size=(4,4))
np.set_printoptions(formatter={'float': '{:.3e}'.format})
text = str(array)
print(text)
#Pour créer les espacement intermédiaires, il faut détecter ceux qui sont entre deux chiffres
#avec éventuellement un signe "-" devant, un simple rechercher/remplacer aurait ajouté des colonnes au début et à la fin
makeTable = re.compile(r"(\d)\s{1,}(-?\d)")
text1 = makeTable.sub(r"\1 & \2",text)
#print(text1)
#Maintenant, un simple rechercher/remplacer suffit, donc on peut s'en contenter
text2 = text1.replace("[[","\\begin{pmatrix}\n ").replace("]]","\\\\\n\\end{pmatrix}").replace("[","").replace("]",r"\\")
#print(text2)
replaceSciLatex = re.compile(r"(\d*)[\.,](\d*)e(-?)+?0?(\d*)")
text3 = replaceSciLatex.sub(r"\1,\2\\cdot 10^{\3\4}",text2)
#On corrige pour les puissances de dix nulles
text3 = text3.replace(r"\cdot 10^{}+00","")
print(text3)

[[-1.561e+00 9.122e-02 2.325e+00 -1.477e-01]
 [-6.084e-01 2.917e-01 1.695e+00 -6.156e-01]
 [-9.595e-01 9.595e-01 4.453e-01 1.249e+00]
 [-1.277e+00 1.303e+00 2.917e-01 -1.641e+00]]
\begin{pmatrix}
 -1,561 & 9,122\cdot 10^{-2} & 2,325 & -1,477\cdot 10^{-1}\\
 -6,084\cdot 10^{-1} & 2,917\cdot 10^{-1} & 1,695 & -6,156\cdot 10^{-1}\\
 -9,595\cdot 10^{-1} & 9,595\cdot 10^{-1} & 4,453\cdot 10^{-1} & 1,249\\
 -1,277 & 1,303 & 2,917\cdot 10^{-1} & -1,641\\
\end{pmatrix}
